In [1]:
import tensorflow as tf
from tensorflow                              import keras
from tensorflow.keras                        import layers, Sequential
from tensorflow.keras.preprocessing.text     import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics         import confusion_matrix

import pandas as pd
import numpy  as np

import matplotlib.pyplot as plt

import os
import time

import nltk
from nltk.corpus import stopwords

In [2]:
# https://www.kaggle.com/c/nlp-getting-started: NLP Disaster Tweets dataset
df = pd.read_csv("train.csv")

In [3]:
df.shape

(7613, 5)

In [4]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
print((df.target == 1).sum()) # Disaster
print((df.target == 0).sum()) # No Disaster

3271
4342


In [6]:
# Preprocessing
import re
import string

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

# https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate/34294022
def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [7]:
pattern = re.compile(r"https?://(\S+|www)\.\S+")

for t in df.text:
    matches = pattern.findall(t)
    for match in matches:
        print(t)
        print(match)
        print(pattern.sub(r"", t))
    if len(matches) > 0:
        break

@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
t
@bbcmtd Wholesale Markets ablaze 


In [8]:
df["text"] = df.text.map(remove_URL)
df["text"] = df.text.map(remove_punct)

In [9]:
# remove stopwords
nltk.download('stopwords')


# Stop Words: A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine
# has been programmed to ignore, both when indexing entries for searching and when retrieving them 
# as the result of a search query.
stop = set(stopwords.words("english"))

# https://stackoverflow.com/questions/5486337/how-to-remove-stop-words-using-nltk-or-python
def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to /Users/Andy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# stop

In [12]:
df["text"] = df.text.map(remove_stopwords)

In [13]:
df.text

0            deeds reason earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       13000 people receive wildfires evacuation orde...
4       got sent photo ruby alaska smoke wildfires pou...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    ariaahrary thetawniest control wild fires cali...
7610                      m194 0104 utc5km volcano hawaii
7611    police investigating ebike collided car little...
7612    latest homes razed northern california wildfir...
Name: text, Length: 7613, dtype: object

In [14]:
from collections import Counter

# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


counter = counter_word(df.text)

In [15]:
len(counter)

17971

In [16]:
#counter

In [17]:
counter.most_common(5)

[('like', 345), ('im', 299), ('amp', 298), ('fire', 250), ('get', 229)]

In [18]:
num_unique_words = len(counter)
print(num_unique_words)

17971


### Creating X, y data

In [19]:
X = df.text.to_numpy()
y = df.target.to_numpy()

### Splitting the data into Training and testing

In [152]:
# TODO: Create a test train set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [153]:
#print(X_train)

In [154]:
# Split dataset into training and validation set
train_size = int(X_train.shape[0] * 0.8)

train_df = X_train[:train_size]
val_df   = X_train[train_size:]

#print(train_df)

In [155]:
# split text and labels
train_sentences = train_df
train_labels    = y_train[:train_size]
val_sentences   = val_df
val_labels      = y_train[train_size:]

In [156]:
#train_sentences.shape, val_sentences.shape

In [157]:
test_sentences = X_test
test_labels    = y_test

In [158]:
test_sentences.shape

(2513,)

In [159]:
# Tokenize
# vectorize a text corpus by turning each text into a sequence of integers
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences) # fit only to training

In [160]:
# each word has unique index
word_index = tokenizer.word_index

In [161]:
#word_index

In [162]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences  = tokenizer.texts_to_sequences(test_sentences)
val_sequences   = tokenizer.texts_to_sequences(val_sentences)

In [163]:
#print(train_sentences[10:15])
#print(train_sequences[10:15])

In [164]:
# Pad the sequences to have the same length
# Max number of words in a sequence
max_length = 20

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
val_padded   = pad_sequences(val_sequences, maxlen=max_length, padding="post", truncating="post")

In [165]:
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post", truncating="post")

In [166]:
#train_padded.shape, test_padded.shape, val_padded.shape

In [167]:
#train_padded[10]

In [168]:
#print(train_sentences[10])
#print(train_sequences[10])
#print(train_padded[10])

In [169]:
# Check reversing the indices

# flip (key, value)
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])

In [170]:
#reverse_word_index

In [171]:
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])

In [172]:
decoded_text = decode(train_sequences[10])

#print(train_sequences[10])
#print(decoded_text)

## Shared Metrics

In [173]:
loss    = keras.losses.BinaryCrossentropy(from_logits=False)
optim   = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

## Creating the LSTM

In [204]:
no_of_cells = 32

In [205]:
LSTM_model = Sequential()
LSTM_model.add(layers.Embedding(num_unique_words, 32,input_length=max_length))

# The layer will take as input an integer matrix of size (batch, input_length),
# and the largest integer (i.e. word index) in the input should be no larger than num_words (vocabulary size).
# Now model.output_shape is (None, input_length, 32), where `None` is the batch dimension.

LSTM_model.add(layers.LSTM(no_of_cells, dropout=0.1, return_sequences=True))
LSTM_model.add(layers.LSTM(no_of_cells, dropout=0.1, return_sequences=True))
LSTM_model.add(layers.LSTM(no_of_cells, dropout=0.1, return_sequences=True))
LSTM_model.add(layers.LSTM(no_of_cells, dropout=0.1))
LSTM_model.add(layers.Dense(1, activation="sigmoid"))

LSTM_model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 20, 32)            575072    
_________________________________________________________________
lstm_32 (LSTM)               (None, 20, 32)            8320      
_________________________________________________________________
lstm_33 (LSTM)               (None, 20, 32)            8320      
_________________________________________________________________
lstm_34 (LSTM)               (None, 20, 32)            8320      
_________________________________________________________________
lstm_35 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 33        
Total params: 608,385
Trainable params: 608,385
Non-trainable params: 0
_______________________________________________

In [206]:
LSTM_model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [207]:
start_time = time.time()

LSTM_model.fit(train_padded, train_labels, epochs=20, validation_data=(val_padded, val_labels), verbose=1)

running_time = time.time() - start_time

Epoch 1/20
128/128 [==============================] - 5s 40ms/step - loss: 0.5447 - accuracy: 0.7267 - val_loss: 0.4845 - val_accuracy: 0.7882
Epoch 2/20
128/128 [==============================] - 4s 32ms/step - loss: 0.2311 - accuracy: 0.9145 - val_loss: 0.5797 - val_accuracy: 0.7833
Epoch 3/20
128/128 [==============================] - 4s 32ms/step - loss: 0.1232 - accuracy: 0.9613 - val_loss: 0.6703 - val_accuracy: 0.7676
Epoch 4/20
128/128 [==============================] - 4s 30ms/step - loss: 0.0754 - accuracy: 0.9752 - val_loss: 0.8655 - val_accuracy: 0.7559
Epoch 5/20
128/128 [==============================] - 4s 30ms/step - loss: 0.0554 - accuracy: 0.9816 - val_loss: 1.0503 - val_accuracy: 0.7667
Epoch 6/20
128/128 [==============================] - 4s 29ms/step - loss: 0.0483 - accuracy: 0.9811 - val_loss: 0.9578 - val_accuracy: 0.7824
Epoch 7/20
128/128 [==============================] - 4s 30ms/step - loss: 0.0435 - accuracy: 0.9838 - val_loss: 1.1931 - val_accuracy: 0.7471

In [208]:
print("The training took: {:.2f} seconds.".format(running_time))

The training took: 83.22 seconds.


# Testing LSTM

In [209]:
# Predictting with the training data
LSTM_predictions = LSTM_model.predict(test_padded)
LSTM_predictions = [1 if p > 0.5 else 0 for p in LSTM_predictions]

In [210]:
#print(test_sentences[10:20])

#print(test_labels[10:20])
#print(LSTM_predictions[10:20])

In [211]:
confusion_matrix(LSTM_predictions, test_labels)

array([[1216,  335],
       [ 230,  732]])

## Creating the GRU

In [212]:
GRU_model = Sequential()
GRU_model.add(layers.Embedding(num_unique_words, 32, input_length=max_length))

GRU_model.add(layers.GRU(no_of_cells, dropout=0.1, return_sequences=True))
GRU_model.add(layers.GRU(no_of_cells, dropout=0.1, return_sequences=True))
GRU_model.add(layers.GRU(no_of_cells, dropout=0.1))
GRU_model.add(layers.Dense(1, activation="sigmoid"))

GRU_model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 20, 32)            575072    
_________________________________________________________________
gru_24 (GRU)                 (None, 20, 32)            6336      
_________________________________________________________________
gru_25 (GRU)                 (None, 20, 32)            6336      
_________________________________________________________________
gru_26 (GRU)                 (None, 32)                6336      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 33        
Total params: 594,113
Trainable params: 594,113
Non-trainable params: 0
_________________________________________________________________


In [213]:
GRU_model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [214]:
start_time = time.time()
GRU_model.fit(train_padded, train_labels, epochs=20, validation_data=(val_padded, val_labels), verbose=1)
running_time = time.time() - start_time

Epoch 1/20
128/128 [==============================] - 4s 32ms/step - loss: 0.6202 - accuracy: 0.6483 - val_loss: 0.5010 - val_accuracy: 0.7725
Epoch 2/20
128/128 [==============================] - 3s 24ms/step - loss: 0.3093 - accuracy: 0.8772 - val_loss: 0.5433 - val_accuracy: 0.7559
Epoch 3/20
128/128 [==============================] - 3s 23ms/step - loss: 0.1540 - accuracy: 0.9478 - val_loss: 0.5922 - val_accuracy: 0.7627
Epoch 4/20
128/128 [==============================] - 3s 23ms/step - loss: 0.0891 - accuracy: 0.9701 - val_loss: 0.7918 - val_accuracy: 0.7676
Epoch 5/20
128/128 [==============================] - 3s 24ms/step - loss: 0.0640 - accuracy: 0.9792 - val_loss: 0.7025 - val_accuracy: 0.7559
Epoch 6/20
128/128 [==============================] - 3s 23ms/step - loss: 0.0518 - accuracy: 0.9828 - val_loss: 0.8953 - val_accuracy: 0.7725
Epoch 7/20
128/128 [==============================] - 3s 23ms/step - loss: 0.0390 - accuracy: 0.9850 - val_loss: 1.0629 - val_accuracy: 0.7608

In [215]:
print("The training took: {:.2f} seconds.".format(running_time))

The training took: 69.52 seconds.


In [216]:
GRU_predictions = GRU_model.predict(test_padded)
GRU_predictions = [1 if p > 0.5 else 0 for p in GRU_predictions]

In [217]:
#print(train_sentences[10:20])

#print(test_labels[10:20])
#print(GRU_predictions[10:20])

In [218]:
confusion_matrix(GRU_predictions, test_labels)

array([[1198,  326],
       [ 248,  741]])